In [30]:
import numpy as np
import pandas as pd

In [31]:
df = pd.read_csv ('https://raw.githubusercontent.com/pycaret/datasets/main/data/common/concrete.csv')
df.columns

Index(['Cement (component 1)(kg in a m^3 mixture)',
       'Blast Furnace Slag (component 2)(kg in a m^3 mixture)',
       'Fly Ash (component 3)(kg in a m^3 mixture)',
       'Water  (component 4)(kg in a m^3 mixture)',
       'Superplasticizer (component 5)(kg in a m^3 mixture)',
       'Coarse Aggregate  (component 6)(kg in a m^3 mixture)',
       'Fine Aggregate (component 7)(kg in a m^3 mixture)', 'Age (day)',
       'strength'],
      dtype='object')

In [32]:
df.columns = ['cement','slag','ash','water','superplastic','coarseAgg','fineAgg','age','strength']
df

,cement,slag,ash,water,superplastic,coarseAgg,fineAgg,age,strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30
...,...,...,...,...,...,...,...,...,...
1025,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28,44.28
1026,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28,31.18
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,23.70
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,32.77


In [33]:
##df.to_csv('../Regression/', index=False)

In [34]:
df.to_csv('./regressionData-Concrete.csv', index=False)

In [35]:
## df.to_csv('../../Supervised-Learning/data/concrete.csv', index=False)

In [36]:
## checking for scaling

def minmax(x):
    return pd.Series(index=['min', 'max','mean','cleanQtile'], data=[x.min(), x.max(),x.mean(),x.quantile(q=0.75) - x.quantile(q=0.25)])
df.apply(minmax).T

,min,max,mean,cleanQtile
cement,102.00,540.0,281.167864,157.625
slag,0.00,359.4,73.895825,142.950
ash,0.00,200.1,54.188350,118.300
water,121.80,247.0,181.567282,27.100
superplastic,0.00,32.2,6.204660,10.200
coarseAgg,801.00,1145.0,972.918932,97.400
fineAgg,594.00,992.6,773.580485,93.050
age,1.00,365.0,45.662136,49.000
strength,2.33,82.6,35.817961,22.425


In [37]:
## Remove unseen samples

data = df.sample(frac=0.99,random_state=28)
data_unseen = df.drop(data.index)

print(f'Data for model: {data.shape},\nData for unseen predictions: { data_unseen.shape}')

Data for model: (1020, 9),
Data for unseen predictions: (10, 9)


In [38]:
from pycaret.regression import *
reg = setup(data = data, target='strength', normalize=True, session_id=42)

,Description,Value
0,Session id,42
1,Target,strength
2,Target type,Regression
3,Original data shape,"(1020, 9)"
4,Transformed data shape,"(1020, 9)"
5,Transformed train set shape,"(714, 9)"
6,Transformed test set shape,"(306, 9)"
7,Numeric features,8
8,Preprocess,True
9,Imputation type,simple


In [39]:
compare_models(exclude=['omp','dummy'])

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,3.4067,24.7720,4.9020,0.9077,0.1499,0.1149,0.0270
et,Extra Trees Regressor,3.4224,29.2213,5.2592,0.8915,0.1576,0.1141,0.0610
rf,Random Forest Regressor,3.9082,32.1600,5.5863,0.8804,0.1736,0.1339,0.0620
gbr,Gradient Boosting Regressor,4.2245,33.1190,5.6632,0.8751,0.1762,0.1446,0.0400
dt,Decision Tree Regressor,4.7547,55.5483,7.3864,0.7939,0.2199,0.1603,0.0280
ada,AdaBoost Regressor,6.4129,62.3452,7.8640,0.7695,0.2798,0.2579,0.0400
knn,K Neighbors Regressor,7.2635,90.5846,9.3506,0.6630,0.3063,0.2691,0.0270
ridge,Ridge Regression,8.1408,107.6078,10.3060,0.6016,0.3307,0.3103,0.0270
lr,Linear Regression,8.1338,107.6337,10.3069,0.6014,0.3305,0.3097,0.0220
br,Bayesian Ridge,8.1556,107.6794,10.3102,0.6014,0.3313,0.3113,0.0250


LGBMRegressor(random_state=42)

In [40]:
# model is run on the train dataset
model = create_model('lightgbm')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,3.0693,19.7519,4.4443,0.9269,0.1145,0.0855
1,2.5142,14.0463,3.7478,0.9542,0.1049,0.0764
2,3.1267,16.7283,4.0900,0.9300,0.1486,0.1140
3,4.6627,42.0308,6.4831,0.8591,0.1631,0.1371
4,2.9243,17.2832,4.1573,0.9430,0.1334,0.0970
5,3.1628,19.2498,4.3875,0.9142,0.1521,0.1129
6,3.1475,26.0820,5.1071,0.9132,0.1843,0.1280
7,4.2526,36.5330,6.0443,0.8578,0.1549,0.1223
8,3.3503,24.1388,4.9131,0.9212,0.1648,0.1308


In [41]:
# Training model on test data set
predict_model(model)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,3.1752,20.2916,4.5046,0.9277,0.1574,0.1183


,cement,slag,ash,water,superplastic,coarseAgg,fineAgg,age,strength,prediction_label
714,254.000000,0.000000,0.000000,198.000000,0.0,968.000000,863.000000,90,26.940001,25.039384
715,230.000000,0.000000,118.300003,195.500000,4.6,1029.400024,758.599976,100,35.340000,33.918469
716,213.800003,98.099998,24.500000,181.699997,6.7,1066.000000,785.500000,100,49.970001,47.179640
717,446.000000,24.000000,79.000000,162.000000,11.6,967.000000,712.000000,7,39.299999,43.824018
718,213.500000,0.000000,174.199997,154.600006,11.7,1052.300049,775.500000,3,17.370001,22.764129
...,...,...,...,...,...,...,...,...,...,...
1015,173.500000,50.099998,173.500000,164.800003,6.5,1006.200012,793.500000,28,38.200001,48.739442
1016,149.000000,236.000000,0.000000,176.000000,13.0,847.000000,893.000000,28,32.959999,31.032593
1017,237.500000,237.500000,0.000000,228.000000,0.0,932.000000,594.000000,90,33.119999,46.150469
1018,475.000000,0.000000,0.000000,228.000000,0.0,932.000000,594.000000,365,41.930000,44.842143


In [42]:
# Tune model goes agaist train and test 
tuned_model = tune_model(model)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.9476,18.0401,4.2474,0.9333,0.1149,0.0857
1,2.7105,14.5987,3.8208,0.9524,0.1176,0.0854
2,2.5701,13.5404,3.6797,0.9433,0.1268,0.0952
3,4.8620,49.3233,7.0231,0.8347,0.1793,0.1433
4,2.3385,10.5459,3.2474,0.9652,0.1230,0.0888
5,3.0146,18.0419,4.2476,0.9196,0.1406,0.1059
6,2.7017,23.8066,4.8792,0.9208,0.1795,0.1174
7,4.4088,43.0870,6.5641,0.8323,0.1814,0.1324
8,3.2046,18.2958,4.2774,0.9402,0.1517,0.1272


In [43]:
print(tuned_model)

LGBMRegressor(bagging_fraction=0.8, bagging_freq=3, feature_fraction=0.8,
              learning_rate=0.2, min_child_samples=6, min_split_gain=0.6,
              num_leaves=30, random_state=42, reg_alpha=0.001, reg_lambda=5)


In [44]:
final_model = finalize_model(tuned_model)
final_model

Pipeline(memory=FastMemory(location=C:\Users\juud0\AppData\Local\Temp\joblib),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['cement', 'slag', 'ash', 'water',
                                             'superplastic', 'coarseAgg',
                                             'fineAgg', 'age'],
                                    transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=[],
                                    transformer=SimpleImputer(strategy='most_frequent'))),
                ('normalize', TransformerWrapper(transformer=StandardScaler())),
                ('actual_estimator',
                 LGBMRegressor(bagging_fraction=0.8, bagging_freq=3,
                               feature_fraction=0.8, learning_rate=0.2,
                               min_child_samples=6, min_split_gain=0.6,
                               num_leaves=30, random_state=42, reg_alpha=0.001,
                               reg_lambda=5))])

In [45]:
save_model(final_model, './models/concrete_ltgm')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=C:\Users\juud0\AppData\Local\Temp\joblib),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['cement', 'slag', 'ash', 'water',
                                              'superplastic', 'coarseAgg',
                                              'fineAgg', 'age'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=[],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('normalize', TransformerWrapper(transformer=StandardScaler())),
                 ('actual_estimator',
                  LGBMRegressor(bagging_fraction=0.8, bagging_freq=3,
                                feature_fraction=0.8, learning_rate=0.2,
                                min_child_samples=6, min_split_gain=0.6,
                                num_leaves=30, random_state=42, reg_alpha=0

In [46]:
model_test = load_model('./models/concrete_ltgm')

Transformation Pipeline and Model Successfully Loaded


In [47]:
data = [[332.5,	142.5,	0.0,	228.0,	0.0,	932.0,	594.0,	365]]
#41.05

In [48]:
cols = model_test.feature_names_in_[:-1]
data_df = pd.DataFrame(data, columns=cols)
prediction = model_test.predict(data_df)[0]
prediction

41.35282973394896

In [49]:
batch = data_unseen.iloc[:,:-1]
batch
predictions = model_test.predict(batch)
predictions

array([29.16396672, 68.27886247, 77.07838182, 30.60932282, 31.19140918,
       54.39744264, 29.82951451, 46.99453254, 16.57758829, 31.93897401])

In [50]:
data_unseen['y-hat'] =np.round(predictions,2)
data_unseen['pct_err'] = (abs(data_unseen['strength'] - data_unseen['y-hat'])) / data_unseen['strength']
data_unseen

,cement,slag,ash,water,superplastic,coarseAgg,fineAgg,age,strength,y-hat,pct_err
87,286.3,200.9,0.0,144.7,11.2,1004.6,803.7,3,24.40,29.16,0.195082
140,313.3,262.2,0.0,175.5,8.6,1046.9,611.8,56,64.90,68.28,0.052080
152,362.6,189.0,0.0,164.9,11.6,944.7,755.8,56,77.30,77.08,0.002846
192,233.8,0.0,94.6,197.9,4.6,947.0,852.2,56,27.66,30.61,0.106652
453,190.3,0.0,125.2,166.6,9.9,1079.0,798.9,56,31.72,31.19,0.016709
472,446.0,24.0,79.0,162.0,11.6,967.0,712.0,28,44.42,54.40,0.224674
476,446.0,24.0,79.0,162.0,11.6,967.0,712.0,3,25.02,29.83,0.192246
498,355.0,19.0,97.0,145.0,12.3,967.0,871.0,28,55.45,46.99,0.152570
606,236.0,0.0,0.0,194.0,0.0,968.0,885.0,14,12.84,16.58,0.291277
939,145.9,230.5,0.0,202.5,3.4,827.0,871.8,28,32.72,31.94,0.023839


In [51]:
#y is the target and y-hat is the predictions of y on unsendata

In [52]:
evaluate_model(model_test)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…